# Please note that you need requests package to run this notebook

In [ ]:
import requests
import json
import pprint

server_port = 9000

# Get Oauth2 Access Token specifying existing taskgroup_uuid

In [ ]:
arguments = {
    'client_id': '8c96bf8cea26fa555fa8',
    'client_secret': '4fd1f508b7b03fba6509da4c193157d7a2b20838',
    'grant_type': 'password',
    'username': 'admin',
    'password': 'admin',
    'taskgroup': '5dw2x3jfkftxue5a5izw6yiplbbn4dlo'
}
r = requests.post(
    'http://127.0.0.1:{server_port}/oauth2/access_token/'.format(server_port=server_port), 
    data=arguments
)
pprint.pprint(r.json())
access_token = json.loads(r.text)['access_token']

# Test authenticated url 

In [ ]:
payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ping/auth/'.format(server_port=server_port), 
    params=payload
)
pprint.pprint(r.text)

# Create new patient with existing demographics and ehr uuid

In [ ]:
payload = {'access_token': access_token, 'demographic_uuid': 'afbab3d293b643833d35cb972f26642cf77f1db4'}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
pprint.pprint(r.json())

# Retrieve all patients for taskgroup

In [ ]:
payload = {'access_token': access_token}

r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
data = r.json()
pprint.pprint(data)
if len(data['patients']) > 0:
    patient_uuid = data['patients'][0]['uuid']

# Retrieve patient by uuid

In [ ]:
payload = {'access_token': access_token}

r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.
    format(server_port=server_port, patient_uuid=patient_uuid), 
    params=payload
)
pprint.pprint(r.json())

# Update patient data by uuid

In [ ]:
payload = {'access_token': access_token, 'demographic_uuid': 'a'}

r = requests.put(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.
    format(server_port=server_port, patient_uuid=patient_uuid), 
    params=payload, data=data
)
pprint.pprint(r.json())

# Delete patient by uuid

In [ ]:
payload = {'access_token': access_token}

r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.
    format(server_port=server_port, patient_uuid=patient_uuid), 
    params=payload, data=data
)
pprint.pprint(r.json())